In [89]:
import tkinter as tk
import copy as cp
class SudokuCSP:
    def __init__(self, puzzle):
        self.grid = cp.deepcopy(puzzle)

    def is_valid(self, row, col, num):
        # Check if 'num' is not in the same row and column and num is not in the intersection of the row and column
        for i in range(9):
            if self.grid[row][i] == num or self.grid[i][col] == num:
                return False

        # Check if 'num' is not in the same 3x3 box
        for i in range(3):
            for j in range(3):
                if self.grid[row - row % 3 + i][col - col % 3 + j] == num:
                    return False

        return True

    def get_domain(self, row, col):
        if self.grid[row][col] != 0:
            return None

        domain = set(range(1, 10))
        for i in range(1, 10):
            if not self.is_valid(row, col, i) and i in domain:
                domain.remove(i)
        return domain

    def find_empty_location(self):
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] == 0:
                    return (i, j)
        return None

    def find_empty_location_mrv(self):
        cases = [ (i, j, domain) for i in range(9) for j in range(9) for domain in [self.get_domain(i, j)] if domain is not None]

               
        if cases:
            min_domain = min(cases, key=lambda x: len(x[2]))
            return (min_domain[0], min_domain[1])
        return None

    def count_constraints(self, row, col, num):
        count = 0
        for i in range(9):
            if i != col and self.is_valid(row, i, num):
                count += 1
            if i != row and self.is_valid(i, col, num):
                count += 1
        s = row - row % 3
        t = col - col % 3
        for i in range(3):
            for j in range(3):
                if s + i != row \
                  and t + j != col \
                  and self.is_valid(s + i, t + j, num):
                    count += 1
        return -count

    def get_lcv(self, row, col):
        domain = self.get_domain(row, col)
        if domain is None:
            return None

        lcv = []
        for num in domain:
            count = self.count_constraints(row, col, num)
            lcv.append((num, count))

        lcv = sorted(lcv, key=lambda x: x[1])
        return lcv

    def solve(self):
        count_nodes = 1
        # Find an empty position
        empty_location = self.find_empty_location() 

        # If there is no empty position, the puzzle is solved
        if  empty_location is None:
            return (True, count_nodes)

        row, col = empty_location

        # Try placing a number from 1 to 9
        for num in range(1, 10):
            if self.is_valid(row, col, num):
                # If the placement is valid, update the grid
                self.grid[row][col] = num
                print('-', end='')
                # Recursively try to solve the rest of the puzzle
                is_solved, count =  self.solve()
                count_nodes += count
                if is_solved:
                    return (True, count_nodes)
                # If placing the current number doesn't lead to a solution, backtrack
                self.grid[row][col] = 0

        # If no number leads to a solution, backtrack
        return (False, count_nodes)

    def solve_mrv(self):
        empty_location = self.find_empty_location_mrv() 
        count_nodes = 1
        if  empty_location is None:
            return (True, count_nodes)

        row, col = empty_location

        # choose a value from the domain       
        for num in self.get_domain(row, col):
            self.grid[row][col] = num

            isSolved, count = self.solve_mrv()
            count_nodes += count
            if isSolved:
                return (True, count_nodes)

            self.grid[row][col] = 0

        # If no number leads to a solution, backtrack
        return (False, count_nodes)

    def solve_mrv_lcv(self):
        empty_location = self.find_empty_location_mrv() 
        count_nodes = 1
        if  empty_location is None:
            return (True, count_nodes)

        row, col = empty_location

        # choose the least constraining value from the domain
        lcv = self.get_lcv(row, col)
        for num, _ in lcv:
            self.grid[row][col] = num

            is_solved, count = self.solve_mrv_lcv()
            count_nodes += count
            if is_solved:            
                return (True, count_nodes )

            self.grid[row][col] = 0
        
        return (False, count_nodes)

    def display(self):
        for i in range(9):
            for j in range(9):
                if(j in [3, 6]):
                    print("|", end=" ")
                print(self.grid[i][j], end=" ")
            if i in [2, 5]:
                print("\n------+-------+------",end="")
            print()
        print()
        print()



In [90]:

from time import time
# Example usage:
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 0, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 0, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]


# ----------------- BTR -----------------
sudoku_csp = SudokuCSP(puzzle)
start_time = time()
csp_solved, csp_count = sudoku_csp.solve()
print("CSP Time taken:", time() - start_time)
if csp_solved:
    print("Sudoku solved with csp solver in", csp_count, "nodes.")
    sudoku_csp.display()
else:
    print("No solution found with csp solver.")

    

# ----------------- MRV -----------------
sudoku_csp = SudokuCSP(puzzle)
start_time = time()
mrv_solved, mrv_count = sudoku_csp.solve_mrv()
print("MRV Time taken:", time() - start_time)
if mrv_solved:
    print("Sudoku solved with mrvsolver in", mrv_count, "nodes.")
    sudoku_csp.display()
else:
    print("No solution found with mrvsolver.")


# ----------------- MRV + LCV -----------------
sudoku_csp = SudokuCSP(puzzle)
start_time = time()
mrv_lcv_solved, mrv_lcv_count = sudoku_csp.solve_mrv_lcv()
print("MRV + LCV Time taken:", time() - start_time)
if mrv_lcv_solved:
    print("Sudoku solved with mrv + lcv solver in", mrv_lcv_count, "nodes.")
    sudoku_csp.display()
else:
    print("No solution found with mrv + lcv solver.")

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------